In [1]:
# always import
import sys
from time import time

# numpy & scipy
import numpy as np
import scipy

# sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
from sklearn import neighbors

# Hungarian algorithm
!pip install munkres
from munkres import Munkres
from scipy.optimize import linear_sum_assignment

# visuals
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn.manifold import Isomap, TSNE

# maybe
from numba import jit

In [2]:
from sklearn.metrics.cluster import homogeneity_score, completeness_score, \
        v_measure_score, adjusted_rand_score, adjusted_mutual_info_score 

def evaluation_metrics(true, pred):
    return homogeneity_score(true, pred), completeness_score(true, pred),  \
        v_measure_score(true, pred), adjusted_rand_score(true, pred), adjusted_mutual_info_score(true, pred)

In [3]:
# load MNIST data 
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='mnist/')

In [ ]:
"""
# a randomly sampled smaller subset 
from sklearn.utils import resample
X.insert(X.shape[1], 'y', y) # Add y to last column of X
X = resample(X, n_samples=30000) # Reduce sample size to 7000 (10%)
print(X.shape, y.shape)
#Detach y from X again. 
y = X['y']
X = X.drop(columns=['y'])
print(X.shape, y.shape)
"""

In [4]:
# Normalization
y = np.asarray(list(map(int, y)))
X = np.asarray(X.astype(float))
X = scale(X)
n_digits = len(np.unique(y))

In [5]:
# TODO: Kmeans
from scipy.spatial.distance import cdist 

#Function to implement steps given in previous section
#X: training data, n_digits:10(for MNIST, it is 10), init: init centroid method "PrincipleEigenvectors or ForgySeedMethod ")
def kmeans(x, k, max_iterations=300, e=1e-4, init="ForgySeedMethod"):
    # Step 1: Initialization of K: Randomly choosing centroids from X (the Forgy seed method)
    if init == "ForgySeedMethod": 
        idx = np.random.choice(len(x), k, replace=False)
        centroids = x[idx, :]
    elif init == "PrincipleEigenvectors":
        pca = PCA()
        pca.fit_transform(x)
        centroids = pca.components_[:k,:] # 10 principle eigenvectors
      
    # Step 2 : Assign each data point xi to the closet centroid
    distances = cdist(x, centroids ,'euclidean') # distance between X and centeroids #Cluster Labe: 0-9
    points = np.array([np.argmin(i) for i in distances]) # Centroid with the minimum Distance
     
    # Step 3: update each cluster centroid Cj as the mean of the data points assigned to cluster-j, i.e.
    iter = 0
    objective = 1e+10
    e = 1
    while (iter <= max_iterations) and (e > 1e-4): #if the number of iterations reaches a maximum value T. and (e > 1e-4)
        iter+=1
        centroids = []
        # Find the new mean
        for idx in range(k):
            mean_cent = x[points==idx].mean(axis=0) #Updating Centroids by taking mean of Cluster it belongs to
            centroids.append(mean_cent)
        
        #Objective function
        objective_before = objective
        objective = np.square(cdist(x, centroids ,'euclidean')).sum() #Distance^2 from x to centeroid that x is assigned. 
        e = abs(objective_before - objective)
  
        centroids = np.vstack(centroids) #Updated Centroids 
        distances = cdist(x, centroids ,'euclidean')
        points = np.array([np.argmin(i) for i in distances])
         
    return points, objective, centroids


In [ ]:
#Problem 2(a)-i. [9 points] Use the top-K principle eigenvectors of X achieved by PCA as the the K initial cluster 
#centroids in Step 1, and run your K-means implementation once on the original MNIST dataset X with the 784 raw pixel 
#features. Report the five evaluation metrics.
print("[Problem 2(a)-i]")
print(". Initial cluster: K principle eigenvctors")
print(". Data: original MNIST dataset X with the 784 raw pixel")
label1, objective, centroids = kmeans(X, n_digits, init="PrincipleEigenvectors")
print(". Report the five evaluation metrics: ")
homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info = evaluation_metrics(y, label1)
print(". homogeneity: {:0.2f}, completeness: {:0.2f}, v_measure: {:0.2f}, adjusted_rand: {:0.2f}, adjusted_mutual_info: {:0.2f}"\
      .format(homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info))

In [ ]:
# Problem 2(a)-ii 
# Apply PCA with X'(# of features = 30)
# Run your K-means implementation 10 times on X′
# each with a different random initializations of the cluster centroids in Step 1
# Select the K-means results (out of the 10 runs) with the smallest objective value, and report the five evaluation metrics

In [ ]:
print("[Problem 2(a)-ii]")
print(". Apply PCA with X'(# of features = 30)")
pca = PCA(n_components=30) # Apply PCA with X'(# of features = 30)
pca_X = pca.fit_transform(X)

lowest_objective = 1e9
for _ in range(10):  #Run your K-means implementation 10 times on X′
    label, objective, centroids = kmeans(pca_X, n_digits, init="ForgySeedMethod")
    if (objective < lowest_objective): # Select the K-means results (out of the 10 runs) with the smallest objective value,
        lowest_objective = objective
        homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info = evaluation_metrics(y, label)
        label2 = label

#report the five evaluation metrics
print(". Report the five evaluation metrics: ")
print(". homogeneity: {:0.2f}, completeness: {:0.2f}, v_measure: {:0.2f}, adjusted_rand: {:0.2f}, adjusted_mutual_info: {:0.2f}"\
      .format(homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info))

In [6]:
# TODO: Hungarian algorithm
# (b)Bipartite Graph Matching using the Hungarian Algorithm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def mapping_correct_label(label, y, prt=True):
    confmat = confusion_matrix(label, y)
    #print("Confusion matrix: \n",  confmat)
    matrix = confmat.max() - confmat # Cost is 0 for max value
    #print("Cost matrix: \n", matrix)

    m = Munkres()
    indexes = m.compute(matrix)
    if prt==True:
        print("Achieved assignment(mapping): \n",indexes)

    label_mapped = np.zeros(shape=label.shape, dtype=int)
    for map in indexes:
        label_mapped = label_mapped + np.where(label==map[0], map[1], 0)
    #print("Accuracy score: ", accuracy_score(y, label_mapped))
    return  accuracy_score(y, label_mapped)

In [ ]:
print("[Problem 2(b)-ii]")
print("[Label from 2.1.1]")
accuracy = mapping_correct_label(label1, y)
confmat = confusion_matrix(label1, y)
print("Confusion matrix: \n",  confmat)
print("Accuracy score: ", accuracy)

print("\n[Label from 2.1.2]")
accuracy = mapping_correct_label(label2, y)
confmat = confusion_matrix(label2, y)
print("Confusion matrix: \n",  confmat)
print("Accuracy score: ", accuracy)

In [ ]:
# TODO: Spectral clustering
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

np.set_printoptions(suppress=True)

def spectral_clustering(X, k_value=20, N_clusters=10):
    """
    kNN = NearestNeighbors(n_neighbors=500, algorithm='kd_tree', metric='euclidean')
    X = kNN.fit(X).kneighbors_graph(mode='distance').toarray()
    """
    # Distance Matrix (Disimilarity)
    H = cdist(X, X ,'euclidean')
  
    #Matrix Afinity E
    n=H.shape[0]
    sigma = (1/(n*(n-1)))*np.sqrt(H).sum() 
    E = np.exp(-1*H/(sigma*sigma))

  # diagonal matrix D: diagonal entry equals to the degree of the corresponding data point
    D = np.zeros(shape=E.shape, dtype=float)
    for i in range(X.shape[0]):
        D[i][i] = E[i,:].sum()
  
  # L = I − D−1/2*E*D−1/2 Normalized Lapacian
    D_ = np.zeros(shape=D.shape, dtype=float)
    for i in range(D.shape[0]):
        D_[i][i] = np.reciprocal(np.sqrt(D[i][i]))
    I = np.identity(X.shape[0], dtype=float)
    L = I-np.matmul(np.matmul(D_, E),D_)
  
  # eigenvalue decomposition to L
    vals, vecs = eigsh(L, which='SM', k=k_value)
    V = vecs[:,1:] # remove first column
  
    best_score = 0
    for _ in range(3):  #Run your K-means implementation 10 times on X′
        # K-means algorithm
        kmeans = KMeans(init="k-means++", n_clusters=N_clusters, n_init=N_clusters).fit(V)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        #print(np.unique(labels))
        score = mapping_correct_label(labels, y, prt=False)
        if (score > best_score):
            best_score = score
            label_final = labels
            centroids_final = centroids

    return label_final, centroids_final
  
print("Problem 2(c)-i: Implement spectral clustering")
pca = PCA(n_components=30) # Apply PCA with X'(# of features = 30)
pca_X = pca.fit_transform(X)
labels_SC, centroids_SC = spectral_clustering(pca_X, k_value=20, N_clusters=10)
accuracy = mapping_correct_label(labels_SC, y)
print("Accuracy score: ", accuracy)
print("Confusion matrix: \n", confusion_matrix(y, labels_SC))
print("Report the five evaluation metrics: ")
homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info = evaluation_metrics(y, labels_SC)
print(". homogeneity: {:0.2f}, completeness: {:0.2f}, v_measure: {:0.2f}, adjusted_rand: {:0.2f}, adjusted_mutual_info: {:0.2f}"\
      .format(homogeneity, completeness, v_measure, adjusted_rand, adjusted_mutual_info))

Problem 2(c)-i: Implement spectral clustering


In [ ]:
from sklearn.cluster import SpectralClustering
# k: k neighbors
# N_clusters: # of centroids
def kNN_clustering(X, y, init="kMean", k=3, N_clusters=100):
    if init=="kMean":
    # K-means + kNN
        test = X
        #kmeans = KMeans(init="k-means++", n_clusters=N_clusters, n_init=N_clusters).fit(test) # get 100 centroids
        label, objective, centroids = kmeans(test, N_clusters, init="ForgySeedMethod")
    #centroids = kmeans.cluster_centers_ # trained data for KNN
    elif init=="Random_Selection":
        test = X
        idx = np.random.choice(len(test), N_clusters, replace=False)
        centroids = test[idx, :]
    elif init=="Spectral_Clustering":
        # Spectral Clustering + kNN
        test = X
        labels_SC, centroids = spectral_clustering(test, k_value=test.shape[1]+1, N_clusters=N_clusters) # get 100 centroids

  # We select the sample closest to each of the obtained 100 cluster centroids C
    dist = cdist(test, centroids)
    train_y = []
    for i in range(N_clusters):
        index = np.where(dist[:,i]==np.amin(dist[:,i]))[0][0]
        train_x.append(test[index])
        train_y.append(y[index])
    train_x = np.array(train_x) # Sample closest to cluster
    train_y = np.array(train_y).flatten() # label of sample
  
    # Test: test (69900 data), Train: train (100 cetnroid data)
    dist = cdist(test, train_x, 'euclidean')
  
    pred = []
    for i in range(dist.shape[0]):
        sorted_index = sorted(range(len(dist[i])), key=lambda k: dist[i][k]) #sort by distance. return array index
        sorted_index = sorted_index[0:k] # index of kNN
        kNN_labels = train_y[sorted_index] # labels of kNN
        #print(kNN_labels)
        vote = np.bincount(kNN_labels).argmax() #vote among kNN 
        #print(vote)
        pred.append(vote)

    pred = np.array(pred)
    return pred



In [ ]:
k_kNN=[1,3,5]
for k_ in k_kNN:
    pred = kNN_clustering(pca_X, y, init="kMean",k=k_)
    accuracy = mapping_correct_label(pred, y, prt=False)
    print("Accuracy score for kMean+kNN ({} neighbors): {}".format(k_,accuracy))
for k_ in k_kNN:
    pred = kNN_clustering(pca_X, y, init="Random_Selection",k=k_)
    accuracy = mapping_correct_label(pred, y, prt=False)
    print("Accuracy score for Random_Selection+kNN ({} neighbors): {}".format(k_,accuracy))
for k_ in k_kNN:
    pred = kNN_clustering(pca_X, y, init="Spectral_Clustering",k=k_)
    accuracy = mapping_correct_label(pred, y, prt=False)
    print("Accuracy score for Spectral_Clustering+kNN ({} neighbors): {}".format(k_,accuracy))